In [12]:
from agents import Agent, Runner, function_tool, handoff, ItemHelpers, MessageOutputItem, trace
from dotenv import load_dotenv
import requests
import os

import json

from typing_extensions import TypedDict, Any


In [2]:
load_dotenv()

True

In [3]:
access_token = os.getenv("LINKEDIN_ACCESS_TOKEN")
linkedin_author = os.getenv("LINKEDIN_AUTHOR")

In [4]:
@function_tool
def post_to_linkedin(generated_content: str):
        """
        This function posts content to LinkedIn.

        Args:
            text (str): The content to post.
            visibility (str, optional): The visibility of the post. Defaults to "CONNECTIONS".
        """
        
        # Linkedin API to share content
        share_content_url = "https://api.linkedin.com/v2/ugcPosts"
        
        # Headers with authorization
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
            "X-Restli-Protocol-Version": "2.0.0"
        }
        
        post_data = {
        "author": f"urn:li:person:{linkedin_author}",
        "lifecycleState": "PUBLISHED",
        "specificContent": {
            "com.linkedin.ugc.ShareContent": {
                "shareCommentary": {
                    "text": generated_content
                },
                "shareMediaCategory": "NONE"
            }
        },
        "visibility": {
            "com.linkedin.ugc.MemberNetworkVisibility": "CONNECTIONS"
            }
        }
        
        print(f"Content to post: {generated_content[:50]}...")
        
        response = requests.post(url=share_content_url, headers=headers, json=post_data)

        print(f"LinkedIn API response status code: {response.status_code}")
        print(f"LinkedIn API response: {response.text}")
        
        if response.status_code in [200,201]:
            # Parse and return json response
            return response.json()
        else:
            return {
                "error": response.status_code,
                "message": response.text
            }

In [5]:
# Define Content Generation Agent

with open("prompts/content_generation_prompt.txt") as f:
    content_generation_prompt = f.read()
    
content_generation_agent = Agent(
    name="content_generator",
    instructions=content_generation_prompt,
    model="gpt-4o-mini",
)

print(content_generation_agent)

Agent(name='content_generator', instructions='You are an AI Engineering expert who specializes in creating engaging, professional content for LinkedIn. \nYour expertise spans machine learning, deep learning, MLOps, LLMs, and AI application development.\n\nHelp the user craft compelling LinkedIn posts that:\n- Showcase technical expertise without being overly complex\n- Include relevant industry insights and trends\n- Maintain a professional yet conversational tone\n- Incorporate appropriate hashtags\n- Are formatted appropriately for LinkedIn\'s platform (concise paragraphs, emojis where appropriate)\n- Can include calls-to-action when relevant\n\nWhen asked to generate content, provide thoughtful posts that would resonate with a professional audience in the AI/ML space. \nIf the user provides specific topics or themes, tailor your response accordingly.\n\nAvoid creating content that is overly promotional, contains misinformation about AI technologies, or makes unrealistic claims about

In [6]:
# Define Linkedin Posting Agent 

posting_prompt = """ 
You are a LinkedIn posting assistant.
You will take content from the content generator and post it to LinkedIn.
"""

posting_agent = Agent(
    name="linkedin_poster",
    instructions= posting_prompt,
    model = "gpt-4o-mini",
    tools=[post_to_linkedin]
)



In [18]:
# Define Orchestration Agent

orchestration_prompt = """
Depending on the user's request, you will do the following:
1. Generate content for a LinkedIn post
2. Post the content to LinkedIn

You will use the content generator and the LinkedIn poster agents to accomplish this.

"""

orchestration_agent = Agent(
    name="orchestration_agent",
    instructions=orchestration_prompt,
    model="gpt-4o-mini",
    tools=[
        content_generation_agent.as_tool(
            tool_name="generate_content",
            tool_description="Generate a LinkedIn post",
        ),
        posting_agent.as_tool(
            tool_name="post_to_linkedin",
            tool_description="Post a LinkedIn post",
        )
    ]
)




In [17]:
async def main():
    
    msg = "Generate a LinkedIn post about the importance of networking in the digital age in 1 sentence only."
    
    result =await Runner.run(
        orchestration_agent,
        msg
    )
    
    print(result.final_output)
    

In [19]:
await main()

Content to post: The importance of networking in the digital age ca...
LinkedIn API response status code: 201
LinkedIn API response: {"id":"urn:li:share:7305652075000958976"}
Your LinkedIn post about the importance of networking in the digital age has been successfully shared! If you need anything else, just let me know!
